### Pre-requisitos para que funcione correctamente
<ul>
    <li>pip install ibm_db</li>
    <li> Instalar nueva version DB2 11.5 del <a href="https://w3-01.ibm.com/download/standardsoftware/PC/lang_en/issiCatalogPC.html" target="_blank">pc@catalog</a></li>
    <li> Usar version modificada del .bat setup_cmd para setear las conexiones (la tengo yo)</li>
    <li> Carpeta utils, tiene que tener licencia 11.5 (la tengo yo) y los GSKit nuevos version 8.50.89 al 27/4</li>
    <li> El conection string, además de lo que figura en github --> <a href="https://github.com/ibmdb/python-ibmdb#api" target="_blank">IBM_DB API</a>, tiene que tener los parametros security=ssl,SSLClientKeystoredb=<b>Dir ibmca.kdb</b>;SSLClientKeystoreDBPassword=<b>Psw .kdb (.bat)</b></li>
    <li>hostname, database name, port datos claves que tienen que estar bien en ODBC</li>
</ul>

In [14]:
import ibm_db
import ibm_db_dbi as dbi
import pandas as pd
import numpy as np
import time


start = time.time()

inputfile=r"C:\Users\016434613\Desktop\Todo\input_PyDB2.txt"
datalist = []
SSLClientKeystoredb_path="C:\ProgramData\IBM\DB2\DB2COPY1\DB2\ibmca.kdb"

with open(inputfile,'r') as infile:
    for line in infile:
        data = line.split("=")
        datalist.append(data[1].strip())

uid=datalist[0]
psw=datalist[1]
SSLClientKeystoredb=SSLClientKeystoredb_path
SSLClientKeystoreDBPassword=datalist[2]


''' Usando dbi, podemos usar el metodo de pandas read_sql y tener un dataframe directo del resultado '''

conn=dbi.connect("DATABASE=USIBMVRDP1H;HOSTNAME=usibmvrdp1h.ssiplex.pok.ibm.com;PORT=5520;PROTOCOL=TCPIP;UID="+uid+";PWD="+psw+";Security=SSL;SSLClientKeystoredb="+SSLClientKeystoredb+";SSLClientKeystoreDBPassword="+SSLClientKeystoreDBPassword+";", "", "")

sql = ''' SELECT CTY, DIV, LC, TOLI, MAJ, MINOR, SMIN, LERU, FID, SRC, ACCID, ACCTMO,
             ACCTYEAR, AMTLOC, AMTDLR, AMTLOC_REP, AMTDLR_REP, VOUCHER_NBR,
             OEM_PROD_ID, DESCR1, LOCFLD1
          FROM LRPCC.LADETAIL_PM_D
          WHERE (CTY = '613' AND MAJ = '601' AND LERU IN ('001977')
             AND LC IN ('01', '04')) '''

df2=pd.read_sql(sql,conn)

df_Minor = pd.read_excel(r"C:\Users\016434613\Desktop\Todo\Minor Codes.xlsx",sheet_name="REF (2)",converters={'Minor':str})
df_Minor = df_Minor.loc[:,['Minor','Desc']]
df_Minor = df_Minor.rename(columns={'Minor':'MINOR'})
df2 = df2.merge(df_Minor, on="MINOR", how="left")
df2.to_excel(r"C:\Users\016434613\Desktop\dept1977_exp.xlsx", index=False)

end = time.time()
mins = (end-start)/60.00
if mins < 1:
    mins = end-start
    minsstr = "Report created in: {:.2f} seconds".format(mins) 
    print(minsstr)
else:
    minsstr = "Report created in: {:.2f} minutes".format(mins)
    print(minsstr)

Report created in: 3.03 seconds


In [16]:
table_ = pd.pivot_table(df2, values=['AMTDLR_REP'], index=['Desc'], columns=['ACCTMO'],
                       aggfunc=np.sum, fill_value=0, margins=True, margins_name="Total")
table_.sort_values(by=('AMTDLR_REP','Total'),ascending=False)

AMTDLR_REP            \
ACCTMO                                                     01        02   
Desc                                                                      
Total                                                52737.18  46393.62   
Salaries                                             26228.57  25791.72   
Social Seurity Taxes                                 10618.83   8243.68   
Vacations Accrual                                     9434.73    487.60   
Bonuses                                               2617.52   2738.05   
Variable Pay                                             0.00   4760.46   
Awards                                                1299.80   1195.41   
Welfare                                                855.10   1705.31   
Other Comp                                             675.77    723.12   
TRANSFER FROM NON-I/S PROVIDER - DATA CENTER SE...     171.46   -171.46   
DEFINED CONTRIBUTION EXPENSE                           339.86    323.68   
INT SRVS PROV-WEB DEVL-APP/INFR & GRAPHICS             227.63    228.67   
Deprec IBM equip-Non Wkst                              163.00    198.00   
Overtime                                                 0.00      0.00   
Postretirement Plan                                     61.88     68.07   
NON-WIRELESS REMOTE DSL ACCESS                          38.95     95.62   
Retirement Plan                                          4.08      3.88   
Audio Conferencing Products and Services                 0.00      1.81   

                                                                         
ACCTMO                                                    03      Total  
Desc                                                                     
Total                                               44815.14  143945.94  
Salaries                                            23798.32   75818.61  
Social Seurity Taxes                                 8630.75   27493.26  
Vacations Accrual                                    3958.21   13880.54  
Bonuses                                              2729.37    8084.94  
Variable Pay                                            0.00    4760.46  
Awards                                               1317.13    3812.34  
Welfare                                               958.10    3518.51  
Other Comp                                            682.16    2081.05  
TRANSFER FROM NON-I/S PROVIDER - DATA CENTER SE...   1498.74    1498.74  
DEFINED CONTRIBUTION EXPENSE                          291.49     955.03  
INT SRVS PROV-WEB DEVL-APP/INFR & GRAPHICS            241.51     697.81  
Deprec IBM equip-Non Wkst                             198.00     559.00  
Overtime                                              383.45     383.45  
Postretirement Plan                                    68.07     198.02  
NON-WIRELESS REMOTE DSL ACCESS                         56.34     190.91  
Retirement Plan                                         3.50      11.46  
Audio Conferencing Products and Services                0.00       1.81